Radek posted about this [here](https://www.kaggle.com/competitions/h-and-m-personalized-fashion-recommendations/discussion/309220), and linked to a GitHub repo with the code.

I just transferred that code here to Kaggle notebooks, that's all.

In [ ]:
import numpy as np

def apk(actual, predicted, k=10):
    """
    Computes the average precision at k.

    This function computes the average prescision at k between two lists of
    items.

    Parameters
    ----------
    actual : list
             A list of elements that are to be predicted (order doesn't matter)
    predicted : list
                A list of predicted elements (order does matter)
    k : int, optional
        The maximum number of predicted elements

    Returns
    -------
    score : double
            The average precision at k over the input lists

    """
    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)

def mapk(actual, predicted, k=10):
    """
    Computes the mean average precision at k.

    This function computes the mean average prescision at k between two lists
    of lists of items.

    Parameters
    ----------
    actual : list
             A list of lists of elements that are to be predicted 
             (order doesn't matter in the lists)
    predicted : list
                A list of lists of predicted elements
                (order matters in the lists)
    k : int, optional
        The maximum number of predicted elements

    Returns
    -------
    score : double
            The mean average precision at k over the input lists

    """
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np

# https://www.kaggle.com/c/h-and-m-personalized-fashion-recommendations/discussion/308635
def customer_hex_id_to_int(series):
    return series.str[-16:].apply(hex_id_to_int)

def hex_id_to_int(str):
    return int(str[-16:], 16)

def article_id_str_to_int(series):
    return series.astype('int32')

def article_id_int_to_str(series):
    return '0' + series.astype('str')

class Categorize(BaseEstimator, TransformerMixin):
    def __init__(self, min_examples=0):
        self.min_examples = min_examples
        self.categories = []
        
    def fit(self, X):
        for i in range(X.shape[1]):
            vc = X.iloc[:, i].value_counts()
            self.categories.append(vc[vc > self.min_examples].index.tolist())
        return self

    def transform(self, X):
        data = {X.columns[i]: pd.Categorical(X.iloc[:, i], categories=self.categories[i]).codes for i in range(X.shape[1])}
        return pd.DataFrame(data=data)


def calculate_apk(list_of_preds, list_of_gts):
    # for fast validation this can be changed to operate on dicts of {'cust_id_int': [art_id_int, ...]}
    # using 'data/val_week_purchases_by_cust.pkl'
    apks = []
    for preds, gt in zip(list_of_preds, list_of_gts):
        apks.append(apk(gt, preds, k=12))
    return np.mean(apks)

def eval_sub(sub_csv, skip_cust_with_no_purchases=True):
    sub=pd.read_csv(sub_csv)
    validation_set=pd.read_parquet('data/validation_ground_truth.parquet')

    apks = []

    no_purchases_pattern = []
    for pred, gt in zip(sub.prediction.str.split(), validation_set.prediction.str.split()):
        if skip_cust_with_no_purchases and (gt == no_purchases_pattern): continue
        apks.append(apk(gt, pred, k=12))
    return np.mean(apks)

In [ ]:
import pandas as pd

In [ ]:
%%time

transactions = pd.read_parquet('../input/warmup/transactions_train.parquet')
customers = pd.read_parquet('../input/warmup/customers.parquet')
articles = pd.read_parquet('../input/warmup/articles.parquet')

# sample = 0.05
# transactions = pd.read_parquet(f'data/transactions_train_sample_{sample}.parquet')
# customers = pd.read_parquet(f'data/customers_sample_{sample}.parquet')
# articles = pd.read_parquet(f'data/articles_train_sample_{sample}.parquet')

In [ ]:
test_week = transactions.week.max() + 1
transactions = transactions[transactions.week > transactions.week.max() - 10]

Below is the radek code with unnecessary parts removed

In [ ]:
%%time

c2weeks = transactions.groupby('customer_id')['week'].unique()

c2weeks2shifted_weeks = {}

for c_id, weeks in c2weeks.items():
    c2weeks2shifted_weeks[c_id] = {}
    for i in range(weeks.shape[0]-1):
        c2weeks2shifted_weeks[c_id][weeks[i]] = weeks[i+1]
    c2weeks2shifted_weeks[c_id][weeks[-1]] = test_week

candidates_last_purchase = transactions.copy()

weeks = []
for i, (c_id, week) in enumerate(zip(transactions['customer_id'], transactions['week'])):
    weeks.append(c2weeks2shifted_weeks[c_id][week])
    
candidates_last_purchase.week=weeks

mean_price = transactions \
    .groupby(['week', 'article_id'])['price'].mean()

sales = transactions \
    .groupby('week')['article_id'].value_counts() \
    .groupby('week').rank(method='dense', ascending=False) \
    .groupby('week').head(12).rename('bestseller_rank').astype('int8')

bestsellers_previous_week = pd.merge(sales, mean_price, on=['week', 'article_id']).reset_index()
bestsellers_previous_week.week += 1

### Features

Feature 3: image captions

In [ ]:
import pandas as pd # data processing
import numpy as np
import seaborn as sns
import matplotlib.pylab as plt
import os
plt.style.use("ggplot")
pd.set_option('display.max_columns', 200)

In [ ]:
!pip install fastparquet
import fastparquet

Load captions generated in previous notebook

In [ ]:
captions = pd.read_parquet('/kaggle/input/captions/captioning_parquet_files/parquet_files/captionsAll.parquet')
captions

Now we embed the captions with sentenceTransformer

(I looked at Peter Kirby's code from feature engineering because he did this with the regular descriptions)
link here: https://github.com/LienM/ai-project-23-24/blob/main/PeterKirby/FeatureEngineering.ipynb

In [ ]:
!pip install -U sentence-transformers

In [ ]:
# use sentencetransform to convert captions into integers
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L12-v2')
# model = SentenceTransformer('all-mpnet-base-v2')

In [ ]:
#converting to list of descriptions to be passed to encoder
descriptions = captions['caption'].tolist()
#encoding
embeddings = model.encode(descriptions)

In [ ]:
embeddings.shape

In [ ]:
#PCA so we arent adding 300+ new features - top 50 principle components
from sklearn.decomposition import PCA
caption_PCA = PCA(n_components=50)
principle_components = caption_PCA.fit_transform(embeddings)
print(principle_components.shape)
variances = caption_PCA.explained_variance_ratio_
variance = 0
for value in variances:
    variance += value
print(variance)

If I take the first 50 principle components and add their variance together, I get 84%. This means that by removing 334 principle components, I only lose 16% of the information.

In [ ]:
# add all principal components as features
for i in range(principle_components.shape[1]):
    captions["pc"+str(i)] = principle_components[:,i]
captions

In [ ]:
captions = captions.set_index('article_id')
captions

In [ ]:
data = pd.read_parquet('/kaggle/input/captions/captioning_parquet_files/parquet_files/data_with_captions_and_pc.parquet')

In [ ]:
data = data.drop(['caption', 'pc1', 'pc2'], axis=1)
data

In [ ]:
small_sample = data.head(100)
small_sample

In [ ]:
# join captions with data dataframe
data_captions = data.merge(captions, on='article_id', how='left')
data = data_captions

In [ ]:
# data = data.drop(['caption_y', 'pc1_y', 'pc2_y'], axis=1)
# data

In [ ]:
data.caption = data.caption.fillna('')
data

In [ ]:
print(data['caption'].value_counts()[''])

In [ ]:
data.to_parquet('/kaggle/working/data_with_captions_and_pc50.parquet')

__ __ __ __ end of features

In [ ]:
data = pd.read_parquet('/kaggle/input/captions-pc50/data_with_captions_and_pc50.parquet')

In [ ]:
data.sort_values(['week', 'customer_id'], inplace=True)
data.reset_index(drop=True, inplace=True)

In [ ]:
train = data[data.week != test_week]
test = data[data.week==test_week].drop_duplicates(['customer_id', 'article_id', 'sales_channel_id']).copy()

In [ ]:
train.to_parquet('/kaggle/working/train.parquet')
test.to_parquet('/kaggle/working/test.parquet')

In [ ]:
train = pd.read_parquet('/kaggle/working/train.parquet')
test = pd.read_parquet('/kaggle/working/test.parquet')

In [ ]:
train_baskets = train.groupby(['week', 'customer_id'])['article_id'].count().values

In [ ]:
columns_to_use = ['article_id', 'product_type_no', 'graphical_appearance_no', 'colour_group_code', 'perceived_colour_value_id',
'perceived_colour_master_id', 'department_no', 'index_code',
'index_group_no', 'section_no', 'garment_group_no', 'FN', 'Active',
'club_member_status', 'fashion_news_frequency', 'age', 'postal_code', 
                  # 'bestseller_rank', 
#                   'week_nr', 'days_since_last_purchase', 
                  'pc0', 'pc1', 'pc2', 'pc3', 'pc4', 'pc5', 'pc6', 'pc7', 'pc8', 'pc9', 'pc10', 'pc11', 'pc12', 'pc13', 'pc14', 'pc15', 'pc16', 'pc17', 'pc18', 'pc19', 'pc20', 'pc21', 'pc22', 'pc23', 'pc24', 'pc25', 'pc26', 'pc27', 'pc28', 'pc29', 'pc30', 'pc31', 'pc32', 'pc33', 'pc34', 'pc35', 'pc36', 'pc37', 'pc38', 'pc39', 'pc40', 'pc41', 'pc42', 'pc43', 'pc44', 'pc45', 'pc46', 'pc47', 'pc48', 'pc49'
                  #, 'pc50', 'pc51', 'pc52', 'pc53', 'pc54', 'pc55', 'pc56', 'pc57', 'pc58', 'pc59', 'pc60', 'pc61', 'pc62', 'pc63', 'pc64', 'pc65', 'pc66', 'pc67', 'pc68', 'pc69', 'pc70', 'pc71', 'pc72', 'pc73', 'pc74', 'pc75', 'pc76', 'pc77', 'pc78', 'pc79', 'pc80', 'pc81', 'pc82', 'pc83', 'pc84', 'pc85', 'pc86', 'pc87', 'pc88', 'pc89', 'pc90', 'pc91', 'pc92', 'pc93', 'pc94', 'pc95', 'pc96', 'pc97', 'pc98', 'pc99'
                  ]

In [ ]:
string = "["
for i in range(100):
    string += "'pc" + str(i) + "', "
string += "]"
print(string)

In [ ]:
%%time

train_X = train[columns_to_use]
train_y = train['purchased']

test_X = test[columns_to_use]

In [ ]:
train_X.to_parquet('/kaggle/working/train_X.parquet')

In [ ]:
train_y.to_parquet('/kaggle/working/train_y.parquet')

In [ ]:
del data

In [ ]:
train.head()

# Model training

In [ ]:
from lightgbm.sklearn import LGBMRanker

In [ ]:
ranker = LGBMRanker(
    objective="lambdarank",
    metric="ndcg",
    boosting_type="dart",
    n_estimators=100,
    importance_type='gain',
    verbose=10
)

In [ ]:
%%time

ranker = ranker.fit(
    train_X,
    train_y,
    group=train_baskets,
)

In [ ]:
for i in ranker.feature_importances_.argsort()[::-1]:
    print(columns_to_use[i], ranker.feature_importances_[i]/ranker.feature_importances_.sum())

# Calculate predictions

In [ ]:
%time

test['preds'] = ranker.predict(test_X)

c_id2predicted_article_ids = test \
    .sort_values(['customer_id', 'preds'], ascending=False) \
    .groupby('customer_id')['article_id'].apply(list).to_dict()

bestsellers_last_week = \
    bestsellers_previous_week[bestsellers_previous_week.week == bestsellers_previous_week.week.max()]['article_id'].tolist()

# Create submission

In [ ]:
sub = pd.read_csv('/kaggle/input/h-and-m-personalized-fashion-recommendations/sample_submission.csv')

In [ ]:
%%time
preds = []
for c_id in customer_hex_id_to_int(sub.customer_id):
    pred = c_id2predicted_article_ids.get(c_id, [])
    pred = pred + bestsellers_last_week
    preds.append(pred[:12])

In [ ]:
preds = [' '.join(['0' + str(p) for p in ps]) for ps in preds]
sub.prediction = preds

In [ ]:
sub_name = 'basic_model_submission'
sub.to_csv(f'{sub_name}.csv.gz', index=False)